### 문제 2-2 : 영화 리뷰 감정 분석기
- 출력 파서: EnumOutputParser
- 문제
영화 리뷰 텍스트를 입력받아 감정을 "긍정", "부정", "보통" 중 하나로 분류하는 시스템을 만드세요.
- 요구사항:
    - EnumOutputParser와 Enum 클래스 사용
    - 여러 개의 테스트 리뷰로 검증
    - 감정분류 결과를 깔끔하게 출력
- 테스트 리뷰 예시:
    - "이 영화 정말 재미없어요. 시간 낭비였습니다."
    - "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!"
    - "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."

- 분석실패했을 경우에 사용하는 
OutputFixingParser 와 EnumOutputParser 를 같이 사용하는 것은 선택입니다.

In [ ]:

from enum import Enum
from typing import List, Tuple

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser

# ===== 1) 감정 Enum 정의 =====
class Sentiment(Enum):
    긍정 = "긍정"
    부정 = "부정"
    보통 = "보통"

# ===== 2) 파서 정의 =====
enum_parser = EnumOutputParser(enum=Sentiment)

prompt = PromptTemplate(
    template=(
        "다음 영화 리뷰의 전반적 감정을 분류하라.\n"
        "- 출력은 오직 {format_instructions} 중 하나여야 한다.\n"
        "- 추가 설명, 마크다운, 따옴표 없이 한 단어로만 출력.\n\n"
        "리뷰:\n{review}\n"
    ),
    input_variables=["review"],
    partial_variables={"format_instructions": enum_parser.get_format_instructions()},
)

# ===== 3) LLM 설정 =====
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 파서 자동 복구기: LLM을 사용해 잘못된 출력 형식을 고쳐줌
fixing_parser = OutputFixingParser.from_llm(parser=enum_parser, llm=llm)

def classify_review(review: str) -> Tuple[str, Sentiment]:
    """
    리뷰 -> (원문 출력, 파싱된 Enum) 반환
    1차: EnumOutputParser로 직접 파싱 시도
    실패 시: OutputFixingParser로 복구 파싱
    """
    _prompt = prompt.format(review=review)
    resp = llm.invoke(_prompt).content.strip()

    try:
        parsed = enum_parser.parse(resp)  # "긍정"/"부정"/"보통" 이어야 성공
    except Exception:
        # 형식 오류나 장황한 출력이 나오면 복구 파서로 재시도
        parsed = fixing_parser.parse(resp)

    return resp, parsed

# ===== 4) 테스트 리뷰 =====
test_reviews: List[str] = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
    # 여기에 더 추가 가능
]

# ===== 5) 실행 및 깔끔 출력 =====
print("=== 감정 분석 결과 ===")
for i, rv in enumerate(test_reviews, 1):
    raw, sentiment = classify_review(rv)
    print(f"{i}. 리뷰: {rv}")
    print(f"   모델출력: {raw}")
    print(f"   분류결과: {sentiment.value}")
    print("-" * 60)
